# Inverse Kinematics

Changelog:
- Created by Stephen Bailey, Anant Sahai
- Fall 21: Added draggable goal animation, Ashwin Vangipuram

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib nbagg

# Function Definitions
This computes $\vec f(\vec \theta)$, which is the function that takes the amount each joint will be rotate and gives us the position of the end effector.  This computation is called forward kinematics.

In [ ]:
def f(r, theta, start):
    """
    Compute the forward kinematics.
    
    Args:
        r: array of lengths of each joint of the arm
        theta: array of angles which tell how much each joint should be rotated
        start: 3D location of the first/root joint of the arm
    
    Returns:
        points: array of 2D points of each joint, each row is a point
    """
    points = np.array([start])
    start_hom = np.append(start, [1]) # The one at the end is there for homogeneous coordinates
    curPoint = start_hom
    T = np.eye(3,3)
    
    # here, we go down the arm rotating the current point and moving it down the length of the joint for each joint
    for i in range(0, r.size):
        # rotation + translation, rigid body transformation matrix
        R = np.array([[np.cos(theta[i]), -np.sin(theta[i]), np.cos(theta[i])*r[i]],
                      [np.sin(theta[i]), np.cos(theta[i]), np.sin(theta[i])*r[i]],
                      [0, 0, 1]])
        T = T @ R
        # update the current point to the end of the current joint
        curPoint = T @ start_hom
        points = np.vstack((points, curPoint[None, :2])) # keep track of the set of points so we can plot the arm later
    return points

This computes $\vec g(\vec \theta)$, which is the function that computes the difference between the location of the end effector $\vec f(\vec{\theta})$ and our target point $\vec t$.

In [ ]:
def g(r, theta, start, target):
    """
    Computes error f(theta) - target.
    
    Args:
        r, theta, and start are the same for f
        target: vector indicating the target point we want the end of the arm to be at
    """
    points = f(r, theta, start)
    end_pos = points[-1, :2]
    return end_pos - target

This computes the Jacobian $J_{\vec g}(\vec\theta)$, which is the 2x4 matrix of partial derivatives of the function $\vec g(\vec \theta)$.

In [ ]:
def Jg(r, theta, start, target):
    """
    Computes the Jacobian of g wrt theta, using a numeric approach. 
    Has the same arguments as g.
    """
    epsilon = 1e-6
    J = np.zeros((2, 0))
    for i in range(0, r.size):
        # use a central difference to estimate the derivatives
        t = theta
        t[i] = t[i] + epsilon
        p = g(r, t, start, target)
        t = theta
        t[i] = t[i] - epsilon
        n = g(r, t, start, target)
        deriv = (p - n) / (2 * epsilon)
        J = np.hstack((J, deriv[:, None]))
    return J

This computes the pseudoinverse of the input matrix.  This is the function that you will be filling in.

In [ ]:
def pseudoinverse(A):
    """
    Computes the pseudoinverse of A. 
    Sets small singular values of A to 0 when calculating the inverse.
    """
    # Your code here: (a)
    U, s, VT = #STUDENT, use NumPy to compute the compact SVD of A
    # End Code #
    # Initialize new singular values to all 0
    s_new = np.zeros_like(s)
    for i in range(0, s.size):
        epsilon = 1e-3
        # Your code here: (b) #STUDENT, invert the not-tiny entries of s here
        if np.abs(s[i]) > epsilon:
            s_new[i] =
        # End Code #
    # Your code here: (c)
    A_pinv = #STUDENT, compute the pseudo inverse using U, VT, and the inverted singular values
    # End Code #
    return A_pinv

This computes the direction we need to move $\vec \theta$ to get closer to the target by using Newton's method.  The direction is given by $-J_{\vec g}(\vec \theta)^{\dagger}\vec g(\vec \theta)$.

In [ ]:
def newton(c, J):
    """
    Calculates the direction of descent for Newton's Method.
    
    Args:
        c: the value g(x)
        J: the Jacobian evaluated at x
    """
    delta = -pseudoinverse(J) @ c
    return delta

This updates $\vec \theta$ by adjusting $\eta$ to decide how far to move $\vec \theta$, and we will thus compute the newton step given by $\vec \theta^{(i+1)} = \vec \theta^{(i)}-\eta J_{\vec g}(\vec{\theta)^{\dagger}}\vec g(\vec \theta)$.  To compute $\eta$, we start with $\eta=1$ and test $\vec{g}(\vec \theta^{(i+1)})$ is closer to the target.  If it's not, we divide $\eta$ by 2 and repeat.  If $\eta$ become small enough and we still haven't found a $\vec \theta^{(i+1)}$ that moves the end effector closer to the target, then we stop.

In [ ]:
# this performs one newton step and estimates eta by starting with eta=1
# if the end effector of the arm is moved farther away after this step,
# we divide eta by 2 and then repeat until we move closer or if eta is too small
def update(r, theta, start, target):
    cost = g(r, theta, start, target)
    oldCost = np.linalg.norm(cost)
    J = Jg(r, theta, start, target)
    delta = newton(cost, J)
    eta = 1.0
    while eta > 1e-6:
        cost = g(r, theta + eta * delta, start, target)
        newCost = np.linalg.norm(cost)
        if newCost < oldCost:
            theta = theta + eta * delta
            break
        eta = eta / 2
    return theta

This puts everything together to compute $\vec \theta$ that moves the arm to the target

In [ ]:
def IK(r, thetaStart, start, target, num_iter=8):
    theta = thetaStart
    oldCost = np.linalg.norm(g(r, theta, start, target))
    newCost = oldCost
    iteration = 0
    while newCost <= oldCost and iteration < num_iter:
        oldCost = newCost
        theta = update(r, theta, start, target)
        newCost = np.linalg.norm(g(r, theta, start, target))
        iteration = iteration + 1
    return theta

This is a helper function to visualize the arm

In [ ]:
def plotArm(r, theta, start, target):
    p = f(r, theta, start)
    fig = plt.figure(figsize=(5, 5))
    plt.xlim(-4, 6)
    plt.ylim(-4, 6)
    plt.plot(*target, 'b-o', markersize=12)
    plt.plot(p[:, 0], p[:, 1], 'r-o',linewidth=4, markersize=8)

# Testing

This tests your pseudo inverse function in three cases

In [ ]:
# We define the location of the first arm joint and the length of each joint
start = np.array([0, 0])
r = np.array([1.0, 1.0, 1.0, 1.0])

# Test if the arm can reach a target
theta = np.array([-0.16227303, -0.60693046, 0.72069313, 0.70009674])
target = np.array([2.0, 2.0])
theta = IK(r, theta, start, target)
plotArm(r, theta, start, target)

# Test if the arm can point to a target out of reach
theta = np.array([0.0, 0.0, 0.0, 0.0])
target = np.array([4.0, 4.0])
theta = IK(r, theta, start, target)
plotArm(r, theta, start, target)

# Test how well the pseudoinverse works with small singular values
theta = np.array([0.0, 0.0, 0.0, 0.0]) + (np.random.rand(4) - 0.5) / 1e6
target = np.array([-1, 3])
theta = IK(r, theta, start, target)
plotArm(r, theta, start, target)

This animates the arm reaching for a moving target.  When the target goes out of reach, the arm should point towards the target because that is the closest it can get to the target.

In [ ]:
# initialize the arm
theta_ani = np.array([0.0, 0.0, 0.0, 0.0])
r = np.array([1.0, 1.0, 1.0, 1.0])
start = np.array([0, 0])

# set up the plot
fig_ani, _ = plt.subplots(figsize=(4, 6))
plt.xlim(-1, 4)
plt.ylim(-5, 5)
line_ani, = plt.plot([], [], 'r-o', linewidth=4, markersize=10)
point_ani, = plt.plot([],[],'b-o',markersize=12)
angle = 2.0

def update_arm_ani(num, r, s):
    global angle, theta_ani
    angle = np.mod(angle + 0.05, 2 * np.pi)
    t = np.array([2.0, 4.5 * np.sin(angle)])
    theta_ani = IK(r, theta_ani, s, t)
    theta_ani = np.mod(theta_ani, 2 * np.pi)
    p = f(r, theta_ani, s)
    line_ani.set_data(p.T)
    point_ani.set_data(t)
    return [line_ani, point_ani]

arm_ani = animation.FuncAnimation(fig_ani, update_arm_ani, 200, fargs=(r, start), interval=1000/12, blit=True, repeat=False)

In [ ]:
class DraggableMarker():
    """
    Creates a marker that is draggable.
    
    Args:
        start: start position of the marker. defaults to [0, 0]
        ax: matplotlib.Axes to draw the marker onto
        update_cb: callback function that takes as argument a matplotlib MouseEvent and is called each time an event occurs
    """
    def __init__(self, start=None, ax=None, update_cb=None):
        if ax == None:
            self.ax = plt.gca()
        else:
            self.ax = ax
        self.update_cb = update_cb
        if start is None:
            start = [0, 0]
        self.marker = self.ax.plot(*start, 'b-o', markersize=12)[0]
        self.draggable = False
        
        # Set up event hooks
        self.ax.figure.canvas.mpl_connect("button_press_event", self.click)
        self.ax.figure.canvas.mpl_connect("button_release_event", self.release)
        self.ax.figure.canvas.mpl_connect("motion_notify_event", self.drag)
    
    def click(self, event):
        if event.button == mpl.backend_bases.MouseButton.LEFT:
            # Left click
            self.draggable = True
            self.update(event)
        else:
            self.draggable = False

        self.ax.figure.canvas.draw_idle()        

    def drag(self, event):
        if self.draggable:
            self.update(event)
            self.ax.figure.canvas.draw_idle()

    def release(self,event):
        self.draggable = False
        ax.figure.canvas.draw_idle()

    def update(self, event):
        self.marker.set_data([event.xdata],[event.ydata])
        self.update_cb(event)
        
def update_arm_drag(event):
    global t_drag, theta_drag
    t_drag = np.array([event.xdata, event.ydata])
    theta_drag = IK(r, theta_drag, start, t_drag)
    theta_drag = np.mod(theta_drag, 2 * np.pi)
    p = f(r, theta_drag, start)
    line_drag.set_data(p.T)
    
# initialize the arm
theta_drag = np.array([0.0, 0.0, 0.0, 0.0])
r = np.array([1.0, 1.0, 1.0, 1.0])
start = np.array([0, 0])
t_drag = np.array([0, 1])
theta_drag = IK(r, theta_drag, start, t_drag)
p = f(r, theta_drag, start)

# set up the plot
fig, ax = plt.subplots(figsize=(6, 6))
plt.xlim(-6, 6)
plt.ylim(-6, 6)
line_drag, = plt.plot(p[:, 0], p[:, 1], 'r-o', linewidth=4, markersize=8)

dm = DraggableMarker(t_drag, ax, update_arm_drag)

plt.show()